# EXAMPLE DATA

In [37]:
# means
mn1 = 0.08
mn2 = 0.12
mn3 = 0.15

# std devs
sd1 = 0.15
sd2 = 0.25
sd3 = 0.25

# correlations
c12 = 0.15
c13 = 0.50
c23 = 0.35

# CREATE ARRAYS

In [38]:
import numpy as np 

mn = np.array([mn1, mn2, mn3])
sd = np.array([sd1, sd2, sd3])

corr = np.identity(3)
corr[0, 1] = corr[1, 0] = c12
corr[0, 2] = corr[2, 0] = c13
corr[1, 2] = corr[2, 1] = c23

cov = np.diag(sd) @ corr @ np.diag(sd)

# CHECK THAT THE COVARIANCE MATRIX IS NONSINGULAR

In [39]:
if np.all(np.linalg.eigvals(cov) > 0):
    print("correlations are acceptable")
else:
    print("correlations are inconsistent")

correlations are acceptable


# GENERATE RANDOM PORTFOLIOS

In [40]:
import numpy as np
num_ports = 5000

# first step for generating random portfolios
# don't sum to 1 yet
ports = np.random.uniform(size=(num_ports, 3))

# force some to be on the boundaries to improve the plot
ports[:200, 0] = 0
ports[200:400, 1] = 0
ports[400:600, 2] = 0

# now force to sum to 1
ports = ports / ports.sum(axis=1).reshape(-1, 1)

# PORTFOLIO MEANS AND STD DEVS

In [41]:
port_mn = ports @ mn
port_vr = np.diag(ports @ cov @ ports.T)
port_sd = np.sqrt(port_vr)

# FIGURE

In [42]:
import plotly.graph_objects as go 

string = 'asset 1: %{customdata[0]:.0%}<br>'
string += 'asset 2: %{customdata[1]:.0%}<br>'
string += 'asset 3: %{customdata[2]:.0%}<br>'
string += '<extra></extra>'
trace1 = go.Scatter(
    x=port_sd,
    y=port_mn,
    mode="markers",
    customdata=ports,
    hovertemplate=string
)

trace2 = go.Scatter(
    x=[sd1, sd2, sd3],
    y=[mn1, mn2, mn3],
    text=["Asset 1", "Asset 2", "Asset 3"],
    mode="markers",
    marker=dict(size=15)
)
fig = go.Figure(trace1)
fig.add_trace(trace2)
fig.update_layout(
    xaxis_title="Standard Deviationr",
    yaxis_title="Expected Return",
    xaxis_rangemode = "tozero",
    yaxis_rangemode="tozero",
    xaxis_tickformat=".0%",
    yaxis_tickformat=".0%",
    template="plotly_white",
    showlegend=False
)
fig.show()